Ref:  
- [LLMs: Get predictions from a language model](https://python.langchain.com/en/latest/getting_started/getting_started.html#llms-get-predictions-from-a-language-model)
- [A simple guide to setting the GPT-3 temperature](https://algowriting.medium.com/gpt-3-temperature-setting-101-41200ff0d0be)
- [HumanEval-X: A new benchmark for Multilingual Program Synthesis](https://github.com/THUDM/CodeGeeX/blob/main/codegeex/benchmark/README.md)

## Import HumanEval-X

In [ ]:
# cpp:  164 taske(s), elapsed_time: 2995.141745413188 S, 49.9190290902198 minutes, 0.8319838181703301 hours.
# java: 164 taske(s), elapsed_time: 4135.389279692899 S, 68.92315466154832 minutes, 1.1487192443591387 hours.
# python: 164 taske(s), elapsed_time: 7769.483070801012 S, 129.4913845133502 minutes, 2.15818974188917 hours.

In [ ]:
language = ('cpp', 'java', 'python')[2]

In [ ]:
language

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path('.').resolve().parents[0]))

from codegeex_api import CodeGeeX
from codegeex_utility import stream_jsonl_all

In [ ]:
import json

manage_properties = dict([
        (line.split('=')[0], line.split('=')[1].strip())
            for line in open(f'../../manage.properties')])

print(json.dumps(manage_properties, indent=4, sort_keys=True))

In [ ]:
import os

# Human Evalu Python
HEP_FILE = os.path.join(
        manage_properties['CodeGeeX_home'],
        f'codegeex/benchmark/humaneval-x/{language}' \
                f'/data/humaneval_{language}.jsonl.gz')
assert os.path.exists(HEP_FILE)

HEP = stream_jsonl_all(HEP_FILE)

print(len(HEP))

In [ ]:
LANGUAGE = HEP[0]['task_id'].split('/')[0]

In [ ]:
LANGUAGE

In [ ]:
PROMPT = HEP[0]['prompt']

In [ ]:
PROMPT

In [ ]:
print(PROMPT)

## Ask CodeGeeX to generate code

#### <font size="7" color="orange">⚠</font> Do <span style="color:red">NOT</span> submit the config file to GitHub because of security.

In [ ]:
codegeex_api_config = json.load(
        open('codegeex_api_config.json', 'r', encoding='utf-8'))

In [ ]:
codegeex_api_config

**For "1.x.x.x", return string; for "2.x.x.x", return json.**

In [ ]:
m = CodeGeeX(codegeex_api_config)  # model

In [ ]:
m_return = m(PROMPT)

In [ ]:
print(m_return)

In [ ]:
print(json.dumps(m_return, indent=4, sort_keys=True))

**temperature & top_p**  

Ref:  
- Zheng, Q., Xia, X., Zou, X., Dong, Y., Wang, S., Xue, Y., Wang, Z., Shen, L., Wang, A., Li, Y. and Su, T., 2023. Codegeex: A pre-trained model for code generation with multilingual evaluations on humaneval-x. *arXiv preprint arXiv:2303.17568*.

#### 4.1 Evaluation Settings
Page 11:

*For CodeGeeX in code generation, we use t = 0:2; p = 0:95 for pass@1 and
t = 0:8; p = 0:95 for pass@10 and pass@100 (except for Go and JavaScript, where p = 0:9).*

In [ ]:
m_return = m(PROMPT, temperature=0.2, top_p=0.95)

In [ ]:
print(json.dumps(m_return, indent=4, sort_keys=True))

In [ ]:
# verbose=True will let more info print out to the sever's stderr.
# However, client will not get these info.

m_return = m(PROMPT, verbose=True)

In [ ]:
print(json.dumps(m_return, indent=4, sort_keys=True))

In [ ]:
print(m_return['stdout'][0])

**Run 164 and write to jsonl file**

In [ ]:
import datetime

 JSON list format. Ref: https://github.com/THUDM/CodeGeeX/blob/main/codegeex/benchmark/README.md#evaluation

In [ ]:
file_name = f'results/{language}_t02_p095_{datetime.datetime.now()}' \
        .replace(':', '').replace('-', '').replace('.', '_').replace(' ', '_') \
        + '.jsonl'

In [ ]:
file_name

In [ ]:
import time

In [ ]:
st = time.perf_counter()

with open(file_name, 'w') as f:
    for i in HEP:
        print(i['task_id'], end='\t')
        
        # Generate code
        # use t = 0:2; p = 0:95 for pass@1
        m_return = m(i['prompt'], temperature=0.2, top_p=0.95)
        
        line = {
            'task_id': i['task_id'],
            'prompt': i['prompt'],
            'generation': m_return['stdout'][0].replace('<|endoftext|>', '')
        }

        f.write(json.dumps(line))
        f.write('\n')

et = time.perf_counter()
elapsed_time = et - st
print(f'\n{len(HEP)} taske(s), elapsed_time: ' \
        f'{elapsed_time} S, {elapsed_time/60} minutes, {elapsed_time/60/60} hours.')
